### **Alpaca API Python Wrapper**

In [79]:
import configparser
from datetime import datetime
import time
from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data import CryptoDataStream, StockDataStream
from alpaca.data.requests import StockLatestQuoteRequest, CryptoLatestQuoteRequest
from alpaca.data.requests import StockBarsRequest, CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

In [80]:
#!pip install alpaca-py --user

In [97]:
from typing import Any


class tradeAlpaca(object):
    '''python wrapper class for alpaca trading API'''

    def __init__(self, keys_file):
        '''
        Parameters
        ----------
        keys_file: configuration .cfg file path
        with the following text contained within
        that file:

        [alpaca]
        api_key = laskdflasdkf
        secret_key = lskdfja;lskdfjalskdj
        endpoint = https://yoyoyoyoyoy.com
        '''
        # api key attributes
        self.config = configparser.ConfigParser()
        self.config.read(keys_file)
        self.api_key = self.config['alpaca']['api_key']
        self.secret_key = self.config['alpaca']['secret_key']
        self.endpoint = self.config['alpaca']['endpoint']
        # alpaca data clients
        self.crypto_client = CryptoHistoricalDataClient()
        self.stock_client = StockHistoricalDataClient(self.api_key, self.secret_key)
        # streaming clients
        self.crypto_stream = CryptoDataStream(self.api_key, self.secret_key)
        self.stock_stream = StockDataStream(self.api_key, self.secret_key)

    def get_current_price(self, ticker):
        '''
        gets current price of specified instrument
        Parameters
        ----------
        ticker (str): ticker symbol of stock or crypto
        '''
        try:
            request_params = StockLatestQuoteRequest(symbol_or_symbols=[ticker])
            latest_quote = self.stock_client.get_stock_latest_quote(request_params)
            ask = latest_quote[ticker].ask_price
            bid = latest_quote[ticker].bid_price
            time = latest_quote[ticker].timestamp
            return time, float(bid), float(ask)
        except:
            request_params = CryptoLatestQuoteRequest(symbol_or_symbols=[ticker])
            latest_quote = self.crypto_client.get_crypto_latest_quote(request_params)
            ask = latest_quote[ticker].ask_price
            bid = latest_quote[ticker].bid_price
            time = latest_quote[ticker].timestamp
            return time, float(bid), float(ask)

    def get_historical_prices(self, ticker, start, end, freq='day'):
        '''
        gets historical prices of specified instrument
        parameters
        ----------
        ticker (str): ticker symbol of stock or crypto
        start (str): start date, e.g. "2022-06-01"
        end (str): end date, e.g. "2022-06-04"
        freq (str): frequency (e.g. "day", "hour","minute", "month", "week")
        '''
        timeframe_dict = {'day':TimeFrame.Day, 'hour':TimeFrame.Hour, 'minute':TimeFrame.Minute, 'month':TimeFrame.Month, 'week':TimeFrame.Week}
        try:
            request_params = StockBarsRequest(
                        symbol_or_symbols=[ticker],
                        timeframe=timeframe_dict[freq],
                        start=datetime.strptime(start, '%Y-%m-%d'),
                        end=datetime.strptime(end, '%Y-%m-%d')
                 )
            bars = self.stock_client.get_stock_bars(request_params)
            bars_df = bars.df
            return bars_df
        except:
            request_params = CryptoBarsRequest(
                        symbol_or_symbols=[ticker],
                        timeframe=timeframe_dict[freq],
                        start=datetime.strptime(start, '%Y-%m-%d'),
                        end=datetime.strptime(end, '%Y-%m-%d')
                 )
            bars = self.crypto_client.get_crypto_bars(request_params)
            bars_df = bars.df
            return bars_df

    async def quote_data_handler(self, data: Any):
        print(data)

    def stream_data(self, ticker):
        '''
        stream the data
        '''
        
        try:
            # wss_client = StockDataStream(self.api_key, self.secret_key)
            # wss_client.subscribe_quotes(self.quote_data_handler, ticker)
            # wss_client.run()
            print(rtherht)
        except:
            wss_client = CryptoDataStream(self.api_key, self.secret_key)
            wss_client.subscribe_quotes(self.quote_data_handler, ticker)
            wss_client.run()


In [98]:
trade_inst = tradeAlpaca(keys_file="../../data/alpaca_keys.cfg")

In [99]:
trade_inst.get_current_price('SPY')

(datetime.datetime(2022, 10, 28, 20, 58, 0, 695, tzinfo=datetime.timezone.utc),
 390.12,
 390.25)

In [100]:
trade_inst.get_historical_prices("BTC/USD", start="2022-01-01", end="2022-10-28").head()

open     high      low    close  \
symbol  timestamp                                                       
BTC/USD 2022-01-01 06:00:00+00:00  47218.0  47933.0  46733.0  47033.0   
        2022-01-02 06:00:00+00:00  47087.0  47985.0  46684.0  47086.0   
        2022-01-03 06:00:00+00:00  47107.0  47555.0  45709.0  46139.0   
        2022-01-04 06:00:00+00:00  46191.0  47522.0  45552.0  46310.0   
        2022-01-05 06:00:00+00:00  46310.0  47053.0  42500.0  43026.0   

                                      volume  trade_count          vwap  
symbol  timestamp                                                        
BTC/USD 2022-01-01 06:00:00+00:00   654.7013       3435.0  47287.231933  
        2022-01-02 06:00:00+00:00   782.8290       3839.0  47211.628791  
        2022-01-03 06:00:00+00:00   956.3265       3784.0  46519.493025  
        2022-01-04 06:00:00+00:00   821.2758       4670.0  46416.927260  
        2022-01-05 06:00:00+00:00  2212.1333       9279.0  44489.282863

In [101]:
trade_inst.stream_data("BTC/USD")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [102]:
wss_client = StockDataStream(trade_inst.api_key, trade_inst.secret_key)

# async handler
async def quote_data_handler(data: Any):
    # quote data will arrive here
    print(data)

wss_client.subscribe_quotes(quote_data_handler, "SPY")

wss_client.run()

RuntimeError: asyncio.run() cannot be called from a running event loop